# 2023_08_30

* sparsity loss

현재 TFT 모델에 사용하는 loss는 qunantile loss를 사용중(default)  

$ \max(q \times (y_{real}-y_{\text{pred}}), (1-q) \times (y_{\text{pred}}-y_{real}))$

현재 사용하는 데이터가 sparsity 함  
이는 $y_{pred} = y_{real} = 0$ 이고 이는 계산을 했을 때 $0$이 나오고 이에 대해서 loss에 대한 계산이 이루어지지 않음  

다른 말로 표현을 하면 TN이 너무 많이 발생하고 TN이 많은 쪽으로 학습이 됨   
실제로 예측의 결과가 0을 많이 예측을 하면 원래의 데이터의 분포와 비슷해서 예측 값이 대부분 0이됨(모델이 보수적)  

우리는 여기에서 1로 예측하는 것을 늘려야 하는 상황임  
이를 위해서는 loss를 계산을 할 떄 $y_{real} > 0 , y_{pred} = 0$ 경우인 FP의 정보가 많이 반영되도록 변경  


* add noise

위에서 본 것과 같이 TN이 많은 상황임  
$y_{pred} = y_{real} = 0$이므로 loss를 계산을 하면 0임  
하지만 $y_{real}$ 부분에 noise를 추가한다면 어느정도 학습이 가능할 것으로 예상이 됨  

noise를 추가한 상황에서 학습을 해서 성능이 어느정도 올라가는지 확인  

### sparsity loss

* confusion matrix

confusion matrix를 이용하기 보다는 loss를 변경해서 해결
confusion matrix를 호출할려고 하면 shape이나 dtype이나 device 같은 문제가 많음  

round를 어떻게 줘야할 애매하고 실제로 round를 했다가는 아래와 같이 기울기에서 문제가 생김  

> http://202.31.200.194:8888/view/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Long%20Beach_5/Test/2023_09_03_sparsity%20loss_round.html


* weight loss

다음과 같이 loss를 변경해서 confusion marix에서의 이점을 loss 계산에서도 가져가도록 만듬  


| $real$ | $pred$ | $confusion matrix$ | $real - 1.5*pred$ |
|------|------|------------------|-------------------|
| 0    | 0    | TN               | 0                 |
| 0    | 0.5  |                  | 0.75              |
| 0    | 1    | FP               | 1.5               |
| 1    | 0    | FN               | 1                 |
| 1    | 0.5  |                  | 0.25              |
| 1    | 1    | TP               | 0.5               |

> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Long%20Beach_5/Test/2023_09_03_sparsity%20loss.ipynb#

